In [2]:
from pydantic import BaseModel
import requests
from datetime import datetime

class WeatherData(BaseModel):
    latitude: float
    longitude: float
    forecast_days: int = 3
    past_days: int = 7
    weather: dict = {}

    def fetch(self) -> None:
        """Fetch past, current, and forecast weather from Open-Meteo and save into self.weather."""
        try:
            url = "https://api.open-meteo.com/v1/forecast"
            params = {
                "latitude": self.latitude,
                "longitude": self.longitude,
                "current_weather": True,
                "past_days": self.past_days,
                "forecast_days": self.forecast_days,
                "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max,relative_humidity_2m_mean",
                "timezone": "auto"
            }

            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            current = data.get("current_weather", {})
            daily = data.get("daily", {})
            times = daily.get("time", [])

            today = datetime.now().date().isoformat()

            past_summary = []
            forecast_summary = []

            for i, date in enumerate(times):
                day_summary = (
                    f"{date}: Temp {daily['temperature_2m_min'][i]}-{daily['temperature_2m_max'][i]}°C, "
                    f"Humidity {daily['relative_humidity_2m_mean'][i]}%, "
                    f"Rain {daily['precipitation_sum'][i]}mm, "
                    f"Wind Max {daily['wind_speed_10m_max'][i]} km/h"
                )
                if date < today:
                    past_summary.append(day_summary)
                else:
                    forecast_summary.append(day_summary)

            self.weather = {
                "location": f"Lat: {self.latitude}, Lon: {self.longitude}",
                "past_7_days": past_summary[-self.past_days:],
                "current": {
                    "temp": f"{current.get('temperature', 'N/A')}°C",
                    "wind": f"{current.get('windspeed', 'N/A')} km/h"
                },
                "forecast_next_days": forecast_summary[:self.forecast_days]
            }

        except requests.RequestException as e:
            self.weather = {"error": str(e)}
        except Exception as ex:
            self.weather = {"error": str(ex)}

# Example usage:
weather_instance = WeatherData(latitude=-1.2921, longitude=36.8219)
weather_instance.fetch()
print(weather_instance.weather)



{'location': 'Lat: -1.2921, Lon: 36.8219', 'past_7_days': ['2025-04-15: Temp 17.1-25.0°C, Humidity 75%, Rain 0.5mm, Wind Max 16.5 km/h', '2025-04-16: Temp 16.1-25.5°C, Humidity 70%, Rain 0.1mm, Wind Max 13.7 km/h', '2025-04-17: Temp 16.5-26.6°C, Humidity 68%, Rain 3.3mm, Wind Max 13.8 km/h', '2025-04-18: Temp 16.8-25.7°C, Humidity 75%, Rain 13.1mm, Wind Max 13.5 km/h', '2025-04-19: Temp 17.0-25.6°C, Humidity 77%, Rain 22.7mm, Wind Max 13.1 km/h', '2025-04-20: Temp 17.1-25.9°C, Humidity 78%, Rain 43.4mm, Wind Max 14.8 km/h', '2025-04-21: Temp 16.5-25.3°C, Humidity 77%, Rain 6.8mm, Wind Max 14.4 km/h'], 'current': {'temp': '18.5°C', 'wind': '7.2 km/h'}, 'forecast_next_days': ['2025-04-22: Temp 17.3-22.2°C, Humidity 84%, Rain 5.9mm, Wind Max 14.1 km/h', '2025-04-23: Temp 16.8-23.2°C, Humidity 83%, Rain 11.7mm, Wind Max 15.9 km/h', '2025-04-24: Temp 16.5-24.2°C, Humidity 77%, Rain 1.2mm, Wind Max 18.4 km/h']}


In [ ]:
# Define the state schema
import os
import numpy as np

from pydantic import BaseModel

from typing import Dict, Any, List, Optional

from matplotlib import pyplot as plt

from ultralytics import YOLO

from langchain_core.runnables import RunnableLambda
from langchain_anthropic import ChatAnthropic

from compute import full_image_processing_pipeline, read_image_from_s3



class LocationModel(BaseModel):
    latitude: Optional[float]
    longitude: Optional[float]

class InputStateModel(BaseModel):
    user_id: Optional[str]
    image_key: str
    coordinates: LocationModel

class WeatherDataOutputModel(BaseModel):
    location: Optional[str]
    past_7_days: List[str]
    current: Dict[str, Any]
    forecast_next_days: List[str]

class YoloAnalysisOutputModel(BaseModel):
    status: Optional[str]
    class_labels: List[str]
    scores: List[float]
    bounding_boxes: List[List[float]]
    save_path: Optional[str]

class NDVIOutputModel(BaseModel):
    ndvi_summary: Optional[Dict[str, float]]
    save_path: Optional[str]

class MauiRecommendationModel(BaseModel):
    risk_level: str
    advice: str
    data_summary: Dict[str, Any]

class OutputStateModel(BaseModel):
    user_id: Optional[str]
    yolo_result: Optional[YoloAnalysisOutputModel]
    weather_data: Optional[WeatherDataOutputModel]
    ndvi_result: Optional[NDVIOutputModel]
    recommendation: Optional[MauiRecommendationModel]

# — define the one “State” model that has *all* fields —
class State(InputStateModel, OutputStateModel):
    pass


def check_for_tiff(input_state: InputStateModel):
    if input_state["image_key"].endswith(".tiff"):
        return "NDVI_pipeline"
    else:
        return "YOLO_analysis"



def weather_node(input_state: InputStateModel) -> OutputStateModel:
    latitude = input_state["coordinates"].get("latitude")
    longitude = input_state["coordinates"].get("longitude")
    output_state: OutputStateModel = {"user_id": input_state.get("user_id")}

    if latitude is not None and longitude is not None:
        weather = WeatherData(latitude=latitude, longitude=longitude)
        weather.fetch()
        output_state["weather_data"] = weather.weather

    return output_state

def YOLO_analysis(input_state: InputStateModel) -> OutputStateModel:
    image_path = input_state["image_key"]
    output_state: OutputStateModel = {"user_id": input_state.get("user_id")}

    try:
        model = YOLO("yolo11s-pest-detection/best.pt")
        results = model.predict(image_path, save=True)

        if not results:
            output_state["yolo_result"] = {"status": "No detections"}
            return output_state

        first_result = results[0]
        output_state["yolo_result"] = {
            "status": "Success",
            "class_labels": first_result.names.values(),
            "scores": [float(c) for c in first_result.boxes.conf.tolist()],
            "bounding_boxes": [b.tolist() for b in first_result.boxes.xyxy],
            "save_path": first_result.save_dir,
        }

    except Exception as e:
        output_state["yolo_result"] = {"status": f"Error: {str(e)}"}

    return output_state



def NDVI_analysis(input_state: InputStateModel) -> OutputStateModel:
    RADIOMETRIC_PARAMS = {
        'gain': [0.012] * 5,
        'offset': [0] * 5,
        'sunelev': 60.0,
        'edist': 1.0,
        'Esun': [1913, 1822, 1557, 1317, 1074],
        'blackadjust': 0.01,
        'low_percentile': 1
    }
    NOISE_METHOD = 'median'
    NOISE_KERNEL_SIZE = 3
    SIGMA = 1.0

    output_state: OutputStateModel = {"user_id": input_state.get("user_id")}

    try:
        image_key = input_state["image_key"]
        bucket_name = "qijaniproductsbucket"
        red_band_index = 2
        nir_band_index = 4

        image = read_image_from_s3(bucket_name, image_key)

        ndvi_noise_reduced, _ = full_image_processing_pipeline(
            image,
            RADIOMETRIC_PARAMS,
            detector_type='ORB',
            noise_method=NOISE_METHOD,
            noise_kernel_size=NOISE_KERNEL_SIZE,
            sigma=SIGMA,
            nir_band_index=nir_band_index,
            red_band_index=red_band_index,
            visualize=False,
            use_parallel_noise_reduction=False
        )

        # Save image preview
        image_save_path = f"outputs/ndvi_{os.path.basename(image_key)}.jpg"
        plt.figure(figsize=(10, 8))
        plt.imshow(ndvi_noise_reduced, cmap='RdYlGn')
        plt.colorbar(label='NDVI Value')
        plt.title("NDVI Analysis")
        plt.savefig(image_save_path, dpi=300)
        plt.close()

        # Save NDVI as .npy
        npy_save_path = image_save_path.replace('.jpg', '.npy')
        np.save(npy_save_path, ndvi_noise_reduced)

        ndvi_summary = {
            "min": float(np.min(ndvi_noise_reduced)),
            "max": float(np.max(ndvi_noise_reduced)),
            "mean": float(np.mean(ndvi_noise_reduced))
        }

        output_state["ndvi_result"] = {
            "ndvi_summary": ndvi_summary,
            "save_path": npy_save_path
        }

    except Exception as e:
        output_state["ndvi_result"] = {"error": str(e)}

    return output_state


def Maui(input_state: OutputStateModel) -> MauiRecommendationModel:
    model = ChatAnthropic(
        model_name="claude-3-haiku-20240307",
        temperature=0.3,
        api_key=os.getenv("ANTHROPIC_API_KEY")
    )

    ndvi_summary = input_state.get('ndvi_result', {}).get('ndvi_summary', {'mean': 'No NDVI data'})
    ndvi_image = input_state.get('ndvi_result', {}).get('save_path', 'No NDVI image saved')

    yolo_detection = input_state.get('yolo_result', {}).get('class_labels', ['No pests detected'])
    weather_summary = input_state.get('weather_data', {'summary': 'No weather data'})

    context = f"""
You are an AI assistant for precision farmers.

Here is the collected data:

- Pest Detection: {', '.join(yolo_detection)}
- NDVI Summary: {ndvi_summary}
- NDVI Image Path: {ndvi_image}
- Weather Current Summary: {weather_summary}

Give me:
1. Risk Level: Low / Moderate / High.
2. Advice: Plain actionable language, no jargon.
3. Data Summary: Recap these findings in under 60 words.
"""

    response = model.invoke([{"role": "user", "content": context}])
    response_text = response.content.strip()

    return {
        "risk_level": "Pending LLM extraction",
        "advice": response_text,
        "data_summary": {
            "pests": yolo_detection,
            "ndvi": ndvi_summary,
            "weather": weather_summary,
            "ndvi_image_path": ndvi_image
        }
    }

In [14]:
from langgraph.graph import StateGraph, START, END


# Create the state graph
graph = StateGraph(State)

# Add nodes
graph.add_node("NDVI_pipeline", NDVI_analysis)
graph.add_node("YOLO_analysis", YOLO_analysis)
graph.add_node("weather_node", weather_node)
graph.add_node("Maui", Maui)

# Define edges and conditional branching
graph.add_edge(START, "weather_node")
graph.add_conditional_edges("weather_node", check_for_tiff, ["NDVI_pipeline", "YOLO_analysis"])  # Checks for TIFF and routes accordingly
graph.add_edge("NDVI_pipeline", "YOLO_analysis")
graph.add_edge("YOLO_analysis", "Maui")
graph.add_edge("Maui", END)

# Compile the graph
advisor_graph = graph.compile()

In [21]:
from IPython.display import Image, display

# View
display(Image(advisor_graph.get_graph(xray=1).draw_mermaid_png()))

ValueError: Failed to render the graph using the Mermaid.INK API. Status code: 502.

In [24]:
from graphviz import Source
from IPython.display import display, Image

# Export to DOT
dot = advisor_graph.get_graph(xray=1).to_dot()

# Render inline
src = Source(dot)
display(src)

# Or save to PNG and display
src.render("state_graph", format="png", cleanup=True)
display(Image("state_graph.png"))

AttributeError: 'Graph' object has no attribute 'to_dot'